In [ ]:
import ast
import pandas as pd

def convert_to_list(s):
    return ast.literal_eval(s)

# DataFrameの読み込み
df = pd.read_csv('../results/output_compress.csv', converters={'memory_usage_ma': convert_to_list, 'elapsed_time_lz78': convert_to_list})
# df['bytes_per_character'] = df['memory_usage_cdawg'] / df['memory_usage_text']
# df['edges_per_character'] = df['num_edges'] / df['text_length']
# df['memory_usage_cdawg'] /= 1024.0 * 1024.0
# df['memory_usage_ma'] /= 1024.0 * 1024.0
df['memory_usage_ma_average'] = df['memory_usage_ma'].apply(lambda x: sum(x) / len(x)) / 1024.0 / 1024.0
df['elapsed_time_lz78_average'] = df['elapsed_time_lz78'].apply(lambda x: sum(x) / len(x)) / 1e6
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
import numpy as np

filenames = ['sources', 'dna', 'english']
colors = { 'sources': 'tab:orange', 'dna': 'tab:green', 'english': 'tab:blue' }
lengths = [1 << x for x in range(3, 27)]

df_construct = pd.read_csv('../results/output_construct.csv', converters={'num_iter_bins': convert_to_list})

memory_usage_cdawg = df_construct[df_construct.text_length == df_construct.text_length.max()].groupby('filename')['memory_usage_cdawg'].mean() / 1024.0 / 1024.0

df_g = df.groupby('filename')

fig, ax = plt.subplots(figsize=(8, 4))
for filename, sub_df in df_g:
    # plot the graph of memory_usage_ma_average
    # set ax as log_scale
    ax.set_xscale('log', base=10)
    ax.set_yscale('log', base=10)
        
    # write horizontal bar indicates memory_usage_cdawg
    ax.axhline(y=memory_usage_cdawg[filename], linestyle=':', color=colors[filename])
    ax.plot(sub_df['substr_length'], sub_df['memory_usage_ma_average'], label=filename, color=colors[filename])
ax.set_xlabel('substring length')
ax.set_ylabel('memory usage (MiB)')
ax.legend()
fig.savefig(f'../results/elapsed_time.pdf')
fig.show()


In [ ]:

fig, ax = plt.subplots(figsize=(8, 4))
for filename, sub_df in df_g:
    # plot the graph of memory_usage_ma_average
    # set ax as log_scale
    ax.set_xscale('log', base=10)
    ax.set_yscale('log', base=10)

    ax.plot(sub_df['substr_length'], sub_df['elapsed_time_lz78_average'], label=filename, color=colors[filename])
    # ax.plot(df['substr_length'], df['memory_usage_ma_average'], label=filename)
ax.set_xlabel('substring length')
ax.set_ylabel('elapsed time (s)')
ax.legend()
fig.savefig(f'../results/memory_usage_ma.pdf')
fig.show()